# Monthly Evaporation Data for each Year
We will obtain the evaporation data for each month for each year across each blockgroup. This will then be implemented in the input file as a Timeseries

#### Imports

In [36]:
import os
import sys
import glob
from tqdm import tqdm

#### Download the NetCDF

In [1]:
import wget

In [11]:
url = 'ftp://ftp.cdc.noaa.gov/Datasets/NARR/Monthlies/monolevel/evap.mon.mean.nc'

wget.download(url)

'evap.mon.mean.nc'

#### Convert NetCDF to GeoTIFF

In [12]:
from osgeo import gdal

In [14]:
projection_5070 = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +towgs84=1,1,-1,0,0,0,0 +units=m +no_defs'
warp_options = gdal.WarpOptions(options = '-t_srs \"' + projection_5070 + '\" -of GTiff')

ds = gdal.Warp('./evap.mon.mean.geotiff', 'NETCDF:./evap.mon.mean.nc:evap', options=warp_options)
ds = None

#### Extract the Bands of the GeoTIFF

The NetCDF data starts recording from January 1979 (Band 1). We only want the data from January 1981 to December 2014.

In [23]:
out_dir = './geotiffs/geotiffs/'
if not os.path.exists(out_dir):
    print('Making', out_dir)
    os.makedirs(out_dir)

In [19]:
start_year = 1981
start_month = 1
end_year = 2014
end_month = 12

start_band = (start_year - 1979) * 12 + start_month
end_band = (end_year - 1979) * 12 + end_month
print('Start:', start_band)
print('End:', end_band)

Start: 25
End: 432


In [29]:
in_ds = gdal.Open('./evap.mon.mean.geotiff')

for band in tqdm(range(start_band, end_band+1)):
    outfile = out_dir + str(band) + '.geotiff'
    
    translate_options = gdal.TranslateOptions(options='-b ' + str(band) + ' -of GTiff')
    gdal.Translate(outfile, in_ds, options=translate_options)

100%|██████████| 408/408 [00:25<00:00, 16.30it/s]


Note that some block groups lie in the ocean, this will cause incorrect evaporation data for them as the oceans have higher evaporation rates.

<img src="https://i.imgur.com/F6UIyls.png">

#### Masking the GeoTIFFs
To solve this issue, we will mask the evaporation data using the land mask provided by NARR NCEP.

##### Download the Land Mask File

In [30]:
url = 'ftp://ftp.cdc.noaa.gov/Datasets/NARR/time_invariant/land.nc'
wget.download(url)

'land.nc'

##### Project the Land Mask

In [32]:
projection_5070 = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +towgs84=1,1,-1,0,0,0,0 +units=m +no_defs'
warp_options = gdal.WarpOptions(options = '-t_srs \"' + projection_5070 + '\" -of GTiff')

ds = gdal.Warp('./land.geotiff', 'NETCDF:./land.nc:land', options=warp_options)
ds = None

##### Import Gdal_calc.py
This will be used to mask the files.


In [35]:
gdal_path = '/home/matas/anaconda3/envs/swmm/bin/'
sys.path.insert(0, gdal_path)
import gdal_calc

In [100]:
out_dir = './geotiffs/masked_geotiffs/'
if not os.path.exists(out_dir):
    print('Making', out_dir)
    os.makedirs(out_dir)

In [ ]:
land_mask = './land.geotiff'

for geotiff in glob.glob('./geotiffs/geotiffs/*.geotiff'):
    print(band_number)
    band_number = geotiff[geotiff.rfind('/')+1:geotiff.rfind('.')]
    outfile = out_dir + band_number + '.geotiff'
    
    gdal_calc.Calc('A*B', A=geotiff, B=land_mask, outfile=outfile, format='GTiff', NoDataValue=0)
    

Now, the block groups that were affected have NoData, since we specified NoData = 0.
<img src="https://i.imgur.com/povTQzI.png">

We will interpolate this missing data from data values around the missing data pixels, extending the raster by one pixel in each direction.

#### Data Interpolation

We will interpolate the data in a 3x3 grid around the target pixel. We will only modify a pixel if it has no data, and if it has at least one neighbor with data.

Neighbors directly adjacent to the target pixel (North, East, South, West) will receive a weight of 1.

Neighbors diagonally adjacent to the target pixel (NW, NE, SE, SW) will receive a weight of $\dfrac{1}{\sqrt{2}}$, since they are farther. Note that each pixel in this dataset represents about 32km$^{2}$.

In [102]:
out_dir = './geotiffs/masked_extended_geotiffs/'
if not os.path.exists(out_dir):
    print('Making', out_dir)
    os.makedirs(out_dir)

In [ ]:
import rasterio as rio
import numpy as np

for geotiff in glob.glob('./geotiffs/masked_geotiffs/*.geotiff'):
    name = geotiff[geotiff.rfind('/')+1:geotiff.rfind('.')]
    print(name)
    raster = rio.open(geotiff)
    data = raster.read(1)
    
    rows = data.shape[0]
    cols = data.shape[1]
    nodata = raster.nodatavals
    data = np.ma.masked_equal(data, nodata)
    output = np.copy(data)
    
    for i in range(rows):
        for j in range(cols):
            if data.mask[i][j]:
                try:
                    total = 0
                    count = 0
                    for x in range(-1,2):
                        for y in range(-1,2):
                            if not data.mask[i+x][j+y]:
                                if (x == -1 or x == 1) and (y == -1 or y == 1):
                                    total += data[i+x][j+y] / 1.41421356237
                                else:
                                    total += data[i+x][j+y]
                                count+= 1
                                
                    output[i][j] = total / count
                except: # Array index error
                    pass
                
                
    with rio.open(out_dir + name + '.geotiff', 'w', **raster.profile) as dst:
        dst.write(output, 1)
    

#### Interpolation Result
<img src="https://i.imgur.com/gZSiuIs.png">

#### Zonal Stats
We will now load the block group shapefile and use rasterstats' zonal_stats function to calculate the evaporation data for each block group.

Note that this will take a long time for each file because there are ~150,000 block groups!

In [107]:
import rasterstats as rs
import geopandas as gpd
import pandas as pd

shape_file = '../../../data/input_file_data/SelectBG_all_land_BGID_final.gpkg'
shape_frame = gpd.read_file(shape_file)
shape_frame['GEOID10'] = shape_frame['GEOID10'].astype(str)

In [108]:
out_dir = './evap/'
if not os.path.exists(out_dir):
    print('Making', out_dir)
    os.makedirs(out_dir)

Making ./evap/


In [ ]:
output_statistics = ['mean']

for file in glob.glob('./geotiffs/masked_extended_geotiffs/*.geotiff'):
    band_number = file[file.rfind('/')+1:file.rfind('.')]
    outfile = out_dir + band_number + '.pkl'
    
    stats = rs.zonal_stats(shape_frame, file, stats=output_statistics, all_touched=True)
    frame = pd.DataFrame.from_dict(stats)
    frame = frame.join(shape_frame['GEOID10'])
    frame.to_pickle(outfile)